<a href="https://colab.research.google.com/github/RaviKishoreReddyP/coconut/blob/main/Coconut_Git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi #GPU checking

In [ ]:
!git clone https://github.com/sunsmarterjie/yolov12.git  #yolov12 cloning from github

In [ ]:
%cd yolov12 #creating a folder

In [ ]:
#installing yolov12 flash libs
!wget https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl

In [ ]:
!pip install -r requirements.txt #requirements for yolov12

In [ ]:
!pip install -e .#requirement for yolov12

In [ ]:
#note: after downloading dataset, need put file path for test, train and valid in data.yaml file

In [ ]:
#group and grayscale coconut data set
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4cLFw28gjLyAqMJ0TBbh")
project = rf.workspace("ravi-mgvlz").project("coconut-group-dataset")
version = project.version(1)
dataset = version.download("yolov12")

In [ ]:
#group coconut data set
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4cLFw28gjLyAqMJ0TBbh")
project = rf.workspace("ravi-mgvlz").project("coconut-group-dataset")
version = project.version(2)
dataset = version.download("yolov12")

In [ ]:
#updated dataset
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4cLFw28gjLyAqMJ0TBbh")
project = rf.workspace("ravi-mgvlz").project("coconut-custom-dataset")
version = project.version(3)
dataset = version.download("yolov12")

In [ ]:
#latest dataset before update
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4cLFw28gjLyAqMJ0TBbh")
project = rf.workspace("ravi-mgvlz").project("coconut-custom-dataset")
version = project.version(1)
dataset = version.download("yolov12")

In [ ]:
#old dataset
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="cUdxg05hwuRayWQqviUd")
project = rf.workspace("niteesh").project("coconut-genre")
version = project.version(2)
dataset = version.download("yolov12")

In [ ]:
#oldest 29 img
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4cLFw28gjLyAqMJ0TBbh")
project = rf.workspace("ravi-mgvlz").project("coconut-dataset-ysc2b")
version = project.version(1)
dataset = version.download("yolov12")

In [ ]:
#trainig in YOLO model note: need to define path at data="file path of data.yaml"
!yolo task=detect mode=train model=yolo12n.pt data=/content/yolov12/coconut-custom-dataset-3/data.yaml epochs=100 imgsz=640

In [ ]:
#validation for training results
!yolo task=detect mode=val model=/content/yolov12/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
#requirements for openCV
!pip install cvzone
!apt-get install libarchive-dev
!pip install cartopy
import cartopy
!pip install matplotlib-venn
!pip install filterpy==1.4.5

In [ ]:
#tracker code

import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [ ]:
#importing all the requirements
import cv2
import pandas as pd
from ultralytics import YOLO
model=YOLO("/content/yolov12/runs/detect/train/weights/best.pt")
from google.colab.patches import cv2_imshow
import cv2
import pandas as pd

In [ ]:
#Detection and Counting code
# Initialize the tracker and other variables
tracker = Tracker()
counter_ids = set()
count = 0
line_y = 1100 # line = high value down low value up
cap = cv2.VideoCapture('/content/InShot_20250429_174309296.mp4') #input file path

# Get video properties to save the output video in the same format as input video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create a VideoWriter object to save the output video
output_video = cv2.VideoWriter('/content/output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

class_list = model.names

while True:
    ret, frame = cap.read()
    if not ret:
        break


    results = model.predict(frame)

    a = results[0].boxes.data
    '''a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")'''
    #px = pd.DataFrame(a.tolist())
    px = pd.DataFrame(a.tolist(), columns=["x_min", "y_min", "x_max", "y_max", "confidence", "class_id"])

    # Extract bounding boxes of class 'coconut'
    bbox_list = [[int(row.x_min), int(row.y_min), int(row.x_max), int(row.y_max)]
                 for _, row in px.iterrows() if 'coconut' in class_list[int(row.class_id)]]

    '''bbox_list = []
    for index, row in px.iterrows():
        x1, y1, x2, y2, conf, d = map(int, row)
        c = class_list[d]
        if 'coconut' in c :
            bbox_list.append([x1, y1, x2, y2])'''

    bbox_id = tracker.update(bbox_list)
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox

        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2
        cv2.circle(frame, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
        cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 0), 1)  # tracker id

        cv2.rectangle(frame, (x3, y3), (x4, y4), (255, 0, 0), 2)

        line_start = (1, line_y)
        line_end = (1200, line_y)
        cv2.line(frame, line_start, line_end, (0, 0, 255), 3)

        if line_y - 15 <= cy <= line_y + 15 and id not in counter_ids:
            counter_ids.add(id)
            count += 1

        cv2.putText(frame, f'total_count : {count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 3)

    # Write the frame to the output video
    output_video.write(frame)

    # Optionally show the frame
    #cv2_imshow(frame)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
      #break

# Release resources
cap.release()
output_video.release()
#cv2.destroyAllWindows()
print("count=", count)
